# MCP Tools

In [ ]:
import nest_asyncio
from IPython.display import Image, display
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

def draw_mermaid_png(agent):
    nest_asyncio.apply()
    display(Image(agent.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))

## Single Agent

### Load configuration

In [ ]:
import json
import os

def load_mcp_servers(config_path):
    """
    Load MCP server definitions from a JSON config file.
    Expects a top-level 'mcpServers' dict in the config.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        config = json.load(f)
    servers = config.get("mcpServers", {})
    # Optionally add default transports if missing
    for name, server in servers.items():
        if "command" in server and "transport" not in server:
            server["transport"] = "stdio"
        if "url" in server and "transport" not in server:
            server["transport"] = "streamable_http"
    return servers

mcp_servers = load_mcp_servers("./mcp_config.json")
mcp_servers

### Define tools

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(mcp_servers)
mcp_tools = await client.get_tools()

In [ ]:
len(mcp_tools)

In [ ]:
mcp_tools[0]

### ReAct Agent

In [ ]:

from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display

GITHUB_AGENT_SYSTEM_MESSAGE = """
You are a GitHub Assistant that helps users manage their GitHub repositories and workflows.

You can help with:
- Repository management (create, fork, browse files)
- Issues and pull requests (create, review, merge)
- Code operations (search, commit, push changes)
- GitHub Actions workflows (run, monitor, debug)
- Notifications and alerts

Use the appropriate GitHub tools based on user requests. 
For complex tasks, break them down into steps and explain what you're doing along the way.

When a user needs help with GitHub, they should simply describe what they want to accomplish, 
and you'll guide them through the process using the available tools.
"""

github_agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=mcp_tools,
    prompt=GITHUB_AGENT_SYSTEM_MESSAGE
)

draw_mermaid_png(github_agent)

### testing

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from textwrap import dedent

prompt = dedent("""
Can you check my account on GitHub and look at my recent work on the langgraph-advanced repo? 
I want to understand what I've been working on lately.
So collect information about all my recent activity and provide a brief overview in natural human language 
about my recent work.
""")

messages = await github_agent.ainvoke({"messages": [HumanMessage(content=prompt)]})
for message in messages['messages']:
    message.pretty_print()

## Problem

In [ ]:
len(mcp_tools)

## Supervisor with smaller Agents

In [ ]:
print(mcp_tools[0].name)
print(mcp_tools[0].description)

In [ ]:
from typing import List, Dict
from pydantic import BaseModel, Field

class AgentDefinition(BaseModel):
    name: str = Field(description="Descriptive name of the agent")
    responsibility: str = Field(description="Clear description of what this agent is responsible for")
    system_message: str = Field(description="System message that guides the agent's behavior")
    tools: List[str] = Field(description="List of tool names this agent should have access to")

class AgentDefinitions(BaseModel):
    agents: List[AgentDefinition] = Field(description="List of agent definitions")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import BaseTool

def _normalize_agent_name(agent_name: str) -> str:
    """Convert an agent name to a valid tool name format (snake_case)."""
    return agent_name.replace(" ", "_").lower()


def create_agent_definitions(tools: list[BaseTool]) -> AgentDefinitions:
    # Extract tool information
    tool_info = []
    for tool in tools:
        tool_info.append({
            "name": tool.name,
            "description": tool.description
        })
    
    # Create prompt for the LLM
    prompt = f"""
    You are an expert in designing multi-agent systems. I have a collection of {len(tool_info)} GitHub-related tools 
    that I want to organize into logical agent groups.
    
    Each tool has a name and description. I want you to analyze these tools and group them into 3-4 logical specialized agents based on related functionality and purpose. Focus on creating broader categories that group related functionality together.
    
    For each agent, provide:
    1. A descriptive name
    2. A clear responsibility statement
    3. A concise system message (2-3 sentences) written in SECOND-PERSON perspective (e.g., "You manage GitHub issues..." NOT "I manage GitHub issues...")
    4. A list of tools this agent should have access to
    
    The goal is to create specialized agents that each handle a specific domain of GitHub operations,
    rather than having one agent with too many tools that might get confused.
    
    Here are the available tools:
    {tool_info}
    
    Make sure every tool is assigned to exactly one agent, and the groupings are logical based on related functionality.
    """
    
    # Call the LLM to get agent definitions
    model = ChatOpenAI(model="gpt-4o-mini")
    structured_model = model.with_structured_output(AgentDefinitions)
    
    response = structured_model.invoke(prompt)

    # Normalize agent names in the response
    for agent in response.agents:
        agent.name = _normalize_agent_name(agent.name)
    
    return response

In [ ]:
agent_definitions = create_agent_definitions(mcp_tools).agents

In [ ]:
len(agent_definitions)

In [ ]:
agent_definitions[0]

In [ ]:
from langchain_core.tools import BaseTool
from langgraph.graph.state import CompiledStateGraph

def create_agents(
    agent_definitions: List[AgentDefinition],
    tools: list[BaseTool]
) -> List[CompiledStateGraph]:
    agents = []
    for agent_def in agent_definitions:
        agent_tools = [tool for tool in tools if tool.name in agent_def.tools]
        agent = create_react_agent(
            model="openai:gpt-4o-mini",
            tools=agent_tools,
            prompt=agent_def.system_message,
            name=agent_def.name
        )
        print(agent_def.name)
        print(agent_def.system_message)
        print(agent_def.tools)
        print("-"*40)
        agents.append(agent)
    return agents

In [ ]:
agents = create_agents(agent_definitions, mcp_tools)

In [ ]:
draw_mermaid_png(agents[0])

### Supervisor

In [ ]:
from IPython.display import Image, display
from langgraph_supervisor import create_supervisor
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.tools import tool


# Generate the agent names and responsibilities section
agents_info = "\n".join([f"- {agent.name.lower()}: {agent.responsibility}" for agent in agent_definitions])

supervisor_system_message = f"""
You are a GitHub Assistant Supervisor that coordinates specialized agents to help users with GitHub tasks.

Your agents:
{agents_info}

Your responsibilities:
1. Understand the user's request and determine which agent can best handle it
2. Route the request to the appropriate agent based on the task type
3. Present the agent's response back to the user
4. If a task requires multiple agents, coordinate the handoff between them

When routing:
- Match the user's request to the most relevant agent's responsibility
- If unsure which agent to use, choose the one with the most relevant tools
- For complex requests, break them down into subtasks for different agents

Keep interactions efficient and focused on solving the user's GitHub tasks.
"""

print(supervisor_system_message)

supervisor = create_supervisor(
    agents=agents,
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=supervisor_system_message,
    
    output_mode="full_history"
).compile()

In [ ]:
draw_mermaid_png(supervisor)

### Testing

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from textwrap import dedent

prompt = dedent("""
Can you check my account on GitHub and look at my recent work on the langgraph-advanced repo? 
I want to understand what I've been working on lately.
So collect information about all my recent activity and provide a brief overview in natural human language 
about my recent work.
""")

messages = await supervisor.ainvoke({"messages": [HumanMessage(content=prompt)]})
for message in messages['messages']:
    message.pretty_print()

## Reference Links

**1. Model Context Protocol (MCP) in LangGraph**

https://langchain-ai.github.io/langgraph/agents/mcp/

→ Guide to integrating Model Context Protocol (MCP) with LangGraph agents, covering MCP server connections, tool exposure, and multi-agent coordination with external context sources.

**2. LangChain MCP Adapters: GitHub Repository**

https://github.com/langchain-ai/langchain-mcp-adapters

→ Official repository for LangChain MCP adapters, providing utilities to connect MCP servers with LangChain/LangGraph agents, including installation, usage examples, and integration patterns.